# WeatherAQI MCP Assistant
**`WeatherAQI MCP Assistant`** is an interactive, asynchronous assistant that brings together real-time weather and AQI (Air Quality Index) data using powerful MCP (Model Context Protocol) servers. It seamlessly connects to dedicated weather and AQI tools, then uses a language model to analyze the data and generate clear, actionable health and safety recommendations. This assistant helps users stay informed about environmental conditions and make better decisions for their well-being. Designed with async operations and SSE connections, it’s perfect for modern, event-driven pipelines.

## Workflow
This diagram illustrates how the WeatherAQI MCP Assistant operates end-to-end within an AI PC environment, combining MCP Compliant Servers, a client, and external APIs.

![Workflow](Workflow.png)

## Import necessary packages
Import all the necessary packages and libraries. Apply nest_asyncio to allow nested event loops

In [1]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import os
from fastmcp import Client

## Define the MCP Client class AQI_Weather_Advisor
This class contains definitions of async methods i.e. `get_weather()`, `get_aqi_report()` and `get_health_recommendations()` that calls corresponding FastMCP tools and return results asynchronously. The `extract_text` helper normalizes different output formats.

In [2]:
class AQI_Weather_Advisor:
    """
    AQI_Weather_Advisor is responsible for interacting with Weather, AQI, and LLM MCP servers.

    It provides methods to:
    - Fetch weather information for a given location.
    - Retrieve AQI (Air Quality Index) reports.
    - Generate health and safety recommendations based on weather and AQI data.
    """

    def __init__(self, weather_url: str, aqi_server_url: str, llm_server_url: str):
        """
        Initialize the AQI_Weather_Advisor with URLs for Weather, AQI, and LLM MCP servers.

        Args:
            weather_url (str): URL of the Weather MCP server.
            aqi_server_url (str): URL of the AQI MCP server.
            llm_server_url (str): URL of the LLM MCP server.
        """
        
        self.weather_url = weather_url
        self.aqi_server_url = aqi_server_url
        self.llm_server_url = llm_server_url

    async def get_weather(self, location: str) -> str:
        """
        Retrieve weather data for a given location from the Weather MCP server.

        Args:
            location (str): Name of the location to get weather information for.

        Returns:
            str: Raw weather data response.
        """
        async with Client(f"{self.weather_url}/sse") as client:
            return await client.call_tool("get_weather", {"location": location})

    async def get_aqi_report(self, location: str) -> str:
        """
        Retrieve AQI (Air Quality Index) report for a given location from the AQI MCP server.

        Args:
            location (str): Name of the location to get AQI report for.

        Returns:
            str: AQI report as plain text.
        """
        async with Client(f"{self.aqi_server_url}/sse") as client:
            result = await client.call_tool("get_aqi", {"location": location})
            return self._extract_text(result)

    async def get_health_recommendations(self, weather_report: str, aqi_report: str) -> str:
        """
        Get health and safety recommendations by calling the LLM MCP server.

        Args:
            weather_report (str): Weather report text.
            aqi_report (str): AQI report text.

        Returns:
            str: Health and safety recommendations.
        """
        async with Client(f"{self.llm_server_url}/sse") as client:
            result = await client.call_tool("safety_guidelines", {"weather_report": weather_report , "aqi_report": aqi_report})
            return self._extract_text(result)

    def _extract_text(self, result) -> str:
        """
        Helper method to extract plain text from MCP results.

        Args:
            result (Any): The result returned from an MCP tool call.
                          Can be a list of blocks or a single object.

        Returns:
            str: Extracted text or string representation.
        """
        if isinstance(result, list):
            return "\n".join(block.text for block in result if hasattr(block, "text"))
        elif hasattr(result, "text"):
            return result.text
        return str(result)


## Create an instance of AQI_Weather_Advisor & Run the Interactive Loop
Instantiates the AQI_Weather_Advisor with the local MCP server URLs and continuously prompts the user for a location. Exits gracefully when the user types exit.
- Calls the weather tool and displays the weather report.
- Calls the AQI tool and shows the AQI details.
- Passes the weather and AQI reports to the LLM tool for health guidelines.
- Prints AI-generated health advice.
- Catches and prints any exceptions.


In [3]:
async def main():
    """
    Main entry point of the MCP based AQI_Weather_Health Assistant.

    Continuously prompts the user for a location, fetches weather and AQI data,
    and provides health and safety recommendations until the user chooses to exit.
    """
    agent = AQI_Weather_Advisor("http://localhost:8000" , "http://localhost:8003", "http://localhost:8004")

    while True:
        location = input("\n📍 Enter location to check for Weather and AQI reports (or 'exit' to quit): ").strip()
        if location.lower() == "exit":
            print("👋 Exiting Weather & AQI Assistant.")
            break

        print("\n🌫️ Fetching Weather & AQI data...")
        try:
                   
            weather_raw = await agent.get_weather(location)
            weather_report = weather_raw[0].text if isinstance(weather_raw, list) else str(weather_raw)
            print("\n📡 Weather Report:\n", weather_report)
            
            aqi_report = await agent.get_aqi_report(location)
            print(f"\n📊 AQI Report for '{location}':\n{aqi_report}")

            print("\n💡 Getting health precautions...")
            recommendations = await agent.get_health_recommendations(weather_report, aqi_report)
            print("\n✅ Health & Safety Advice:\n", recommendations)
        except Exception as e:
            print("❌ An error occurred:", str(e))

## Run the MCP-based-AQI-Weather Assistant Application

Runs the main coroutine to execute the AQI-Weather Assistant’s main user interaction loop asynchronously and waits until it finishes.

In [4]:
await main()


📍 Enter location to check for Weather and AQI reports (or 'exit' to quit):  Delhi



🌫️ Fetching Weather & AQI data...

📡 Weather Report:
 📍 Location: Delhi, India
🗺️ Coordinates: 28.65195, 77.23149
🌡️ Temperature: 28.9°C
💨 Wind Speed: 2.6 km/h

📊 AQI Report for 'Delhi':

📍 Location: Delhi, India
🗺️ Coordinates: 28.65195, 77.23149
🌫️ AQI Level: 2 (Fair)

Pollutants (μg/m3):
- CO: 371.2
- NO: 0
- NO2: 21.67
- O3: 35.17
- SO2: 2.59
- PM2.5: 10.37
- PM10: 22.53
- NH3: 16.76


💡 Getting health precautions...

✅ Health & Safety Advice:
 
You are a health assistant. Given this weather and air quality:

Weather Report:
📍 Location: Delhi, India
🗺️ Coordinates: 28.65195, 77.23149
🌡️ Temperature: 28.9°C
💨 Wind Speed: 2.6 km/h

AQI Report:

📍 Location: Delhi, India
🗺️ Coordinates: 28.65195, 77.23149
🌫️ AQI Level: 2 (Fair)

Pollutants (μg/m3):
- CO: 371.2
- NO: 0
- NO2: 21.67
- O3: 35.17
- SO2: 2.59
- PM2.5: 10.37
- PM10: 22.53
- NH3: 16.76


Provide:
1. Overall outdoor safety level. can i go for parasailing based on the weather report?
2. Health risks.
3. Precautions.
4. Special


📍 Enter location to check for Weather and AQI reports (or 'exit' to quit):  exit


👋 Exiting Weather & AQI Assistant.
